In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/battery-electric-vehicles/Battery_Electric_Vehicles.csv')

In [ ]:
data.head()

# **Fast EDA**

In [ ]:
!pip install fasteda


In [ ]:
from fasteda import fast_eda
fast_eda(data)

In [ ]:
data.head()

In [ ]:
data['CO2 Emissions_(g/km)'].unique()

In [ ]:
data['CO2 _Rating'].unique()

In [ ]:
data['Smog_Rating'].unique()

In [ ]:
data['Fuel_Type'].unique()

In [ ]:
df=data.drop(columns=['CO2 Emissions_(g/km)','CO2 _Rating','Smog_Rating','Fuel_Type'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.corr()

#  Outlier Analysis and removal

In [ ]:
df['C_City (kWh/100 km)'].plot(kind='hist', bins=15)

In [ ]:
df['C_City (kWh/100 km)'].plot.box()

In [ ]:
Q1_1=df['C_City (kWh/100 km)'].quantile(0.25)
Q2_1=df['C_City (kWh/100 km)'].quantile(0.75)
QIR_1=Q2_1-Q1_1
QIR_1

In [ ]:
((df['C_City (kWh/100 km)']>(Q2_1+QIR_2*1.5))|(df['C_City (kWh/100 km)']<(Q1_1-QIR_2*1.5))).sum()

In [ ]:
out1=(df['C_City (kWh/100 km)']>(Q2_1+QIR_2*1.5))|(df['C_City (kWh/100 km)']<(Q1_1-QIR_2*1.5))
df[out1]=np.nan

In [ ]:
df['C_Hwy (kWh/100 km)'].plot(kind='hist', bins=10)

In [ ]:
df['C_Hwy (kWh/100 km)'].plot.box()

In [ ]:
df[['C_Hwy (kWh/100 km)','C_City (kWh/100 km)']].plot.box()

In [ ]:
Q1_2=df['C_Hwy (kWh/100 km)'].quantile(0.25)
Q2_2=df['C_Hwy (kWh/100 km)'].quantile(0.75)
QIR_2=Q2_2-Q1_2
QIR_2

In [ ]:
((df['C_Hwy (kWh/100 km)']>(Q2_2+QIR_2*1.5))|(df['C_Hwy (kWh/100 km)']<(Q1_2-QIR_2*1.5))).sum()

In [ ]:
out=(df['C_Hwy (kWh/100 km)']>(Q2_2+QIR_2*1.5))|(df['C_Hwy (kWh/100 km)']<(Q1_2-QIR_2*1.5))
df[out]=np.nan

In [ ]:
df=df.dropna().reset_index(drop=True)


In [ ]:
df.isnull().sum()

In [ ]:
x_ax1 = range(len(df))
plt.plot(x_ax1, df['C_City (kWh/100 km)'], label="C_City (kWh/100 km)", color='c')
plt.plot(x_ax1, df['C_Hwy (kWh/100 km)'], label="C_Hwy (kWh/100 km)", color='b')
#plt.plot(x_ax, Y_test.iloc[:,1], label="y2-test", color='m')
#plt.plot(x_ax, Y_pred[:,1], label="y2-pred", color='r')
plt.legend()
plt.show()

# Loading machine learning packages

In [ ]:
!pip install scikit-multilearn

# Feature selection 

In [ ]:
features = ['Model_Year', 'Make', 'Model','Vehicle Class','Motor_(kW)', 'Transmission','Range_(km)','Recharge_Time (h)' ]
X= pd.get_dummies(df[features])
y= df[['C_City (kWh/100 km)', 'C_Hwy (kWh/100 km)']]

# Train / Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, y, test_size=0.2, random_state=1)
print ('Train set:', X_train.shape,  Y_train.shape)
print ('Test set:', X_test.shape,  Y_test.shape)

# Building the model

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
MultiOutput=MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

In [ ]:
M_model=MultiOutput.fit(X_train,Y_train)

In [ ]:
Y_pred=M_model.predict(X_test)

In [ ]:
Y_test[0:5]

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(Y_test, M_model.predict(X_test))

In [ ]:
mse

In [ ]:
score = M_model.score(X_train, Y_train)
print("Training score:", score)

In [ ]:
score = M_model.score(X_test, Y_test)
print("Test score:", score)

In [ ]:
print("y1 MSE:%.4f" % mean_squared_error(Y_test.iloc[:,0], Y_pred[:,0])) 
print("y2 MSE:%.4f" % mean_squared_error(Y_test.iloc[:,1], Y_pred[:,1]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Y_test.shape

In [ ]:
Y_test.iloc[:, 0]

In [ ]:
#Finally, we'll visualize the results in the plot and check them visually.

x_ax = range(len(X_test))
plt.plot(x_ax, Y_test.iloc[:,0], label="y1-test", color='c')
plt.plot(x_ax, Y_pred[:,0], label="y1-pred", color='b')
#plt.plot(x_ax, Y_test.iloc[:,1], label="y2-test", color='m')
#plt.plot(x_ax, Y_pred[:,1], label="y2-pred", color='r')
plt.legend()
plt.show()

In [ ]:
x_ax2 = range(len(X_test))
plt.plot(x_ax2, Y_test.iloc[:,1], label="y2-test", color='m')
plt.plot(x_ax2, Y_pred[:,1], label="y2-pred", color='r')
plt.legend()
plt.show()

# The End